<a href="https://colab.research.google.com/github/GuidoLorenzetti/TP2-AA2-Lorenzetti_Farias_Garcia/blob/main/Ejercicio%202/EJ2_Train_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico Final - Aprendizaje Automático II


## Integrantes:

*   Lorenzetti, Guido
*   Farias, Augusto
*   Garica, Timoteo


## Problema 2 - Fake News

El objetivo de este ejercicio es construir modelos de generación de texto utilizando redes neuronales que puedan generar noticias ficticias.


## Setup

In [ ]:
import tensorflow as tf

# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Cargar y preparar el dataset

En esta sección, cargamos el dataset `ag_news_subset` usando `tensorflow_datasets` y preparamos los datos de texto para el entrenamiento.

In [ ]:
import tensorflow_datasets as tfds
import numpy as np
import os

# Cargar el dataset de noticias
dataset, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Convertir a texto plano
train_texts = []
for text, label in tfds.as_numpy(train_dataset):
    train_texts.append(text.decode('utf-8'))

# Concatenar todos los textos en uno solo
text = ' '.join(train_texts)
print(f'Texto total: {len(text)} caracteres')

Texto total: 23328241 caracteres


# Crear secuencias de caracteres

En esta sección, creamos un mapeo de caracteres a índices y convertimos los textos en secuencias de índices. También preparamos las secuencias de entrada y salida para el modelo.


In [ ]:
# Crear un mapeo de caracteres a índices
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convertir los caracteres a índices
text_as_int = np.array([char2idx[c] for c in text])

# Crear secuencias de entrada y salida
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Crear lotes de entrenamiento
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Construcción del modelo

Construimos el modelo de RNN usando capas de LSTM y Dense. También definimos las épocas.

In [ ]:
EPOCHS = 50

# Construcción y entrenamiento del modelo

En esta sección, construimos un modelo secuencial de caractéres de TensorFlow que utiliza dos capas GRU (Gated Recurrent Unit) y una capa densa para la generación de texto. Se define una función de pérdida personalizada que utiliza `sparse_categorical_crossentropy` y se compila el modelo con el optimizador Adam. Finalmente, entrenamos el modelo con el dataset preparado y guardamos el modelo entrenado en un archivo.

In [ ]:
# Construir el modelo
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Función de pérdida
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

history = model.fit(dataset, epochs=EPOCHS)

model.save('model_char.h5')

Epoch 1/50
3608/3608 [==============================] - 88s 23ms/step - loss: 1.4579
Epoch 2/50
3608/3608 [==============================] - 85s 23ms/step - loss: 1.1898
Epoch 3/50
3608/3608 [==============================] - 86s 24ms/step - loss: 1.1569
Epoch 4/50
3608/3608 [==============================] - 80s 22ms/step - loss: 1.1472
Epoch 5/50
3608/3608 [==============================] - 79s 22ms/step - loss: 1.1477
Epoch 6/50
3608/3608 [==============================] - 79s 22ms/step - loss: 1.1571
Epoch 7/50
3608/3608 [==============================] - 82s 23ms/step - loss: 1.2028
Epoch 8/50
3608/3608 [==============================] - 83s 23ms/step - loss: 1.7619
Epoch 9/50
3608/3608 [==============================] - 82s 23ms/step - loss: 2.0328
Epoch 10/50
3608/3608 [==============================] - 81s 22ms/step - loss: 1.9515
Epoch 11/50
3608/3608 [==============================] - 79s 22ms/step - loss: 1.7585
Epoch 12/50
3608/3608 [==============================] - 78s 21

# Tokenización y preparación del dataset basado en palabras

Tokenizamos el texto en palabras en lugar de caracteres, y preparamos los datos para el entrenamiento del modelo. Creamos un mapeo de palabras a índices y convertimos el texto en secuencias de índices. Luego, creamos las secuencias de entrada y salida necesarias para el entrenamiento del modelo y las organizamos en lotes.


In [ ]:
import re

# Tokenizar el texto en palabras
words = re.findall(r'\b\w+\b', text.lower())
word2idx = {u: i for i, u in enumerate(set(words))}
idx2word = np.array(list(set(words)))

# Convertir las palabras a índices
text_as_int = np.array([word2idx[w] for w in words])

# Crear secuencias de entrada y salida
seq_length = 20
examples_per_epoch = len(words) // seq_length

word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = word_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Crear lotes de entrenamiento
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Construcción y entrenamiento del modelo basado en palabras

Construimos un modelo secuencial de palabras de TensorFlow que utiliza dos capas GRU (Gated Recurrent Unit) y una capa densa para la generación de texto basado en palabras. Se compila el modelo con el optimizador Adam y la función de pérdida previamente definida. Finalmente, se entrena el modelo con el dataset preparado y se guarda el modelo entrenado en un archivo.


In [ ]:
# Construir el modelo
vocab_size = len(word2idx)
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Función de pérdida
model.compile(optimizer='adam', loss=loss)

# Entrenar el modelo
history = model.fit(dataset, epochs=EPOCHS)

model.save('model_words.h5')

Epoch 1/50
2900/2900 [==============================] - 115s 39ms/step - loss: 8.2471
Epoch 2/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2565
Epoch 3/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2609
Epoch 4/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2602
Epoch 5/50
2900/2900 [==============================] - 114s 39ms/step - loss: 8.2606
Epoch 6/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2604
Epoch 7/50
2900/2900 [==============================] - 114s 39ms/step - loss: 8.2600
Epoch 8/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2594
Epoch 9/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2592
Epoch 10/50
2900/2900 [==============================] - 114s 39ms/step - loss: 8.2615
Epoch 11/50
2900/2900 [==============================] - 113s 39ms/step - loss: 8.2605
Epoch 12/50
2900/2900 [=============================